Create a panel for company, with relevant info

In [15]:
from __future__ import absolute_import, division, print_function
import datetime

import pandas as pd


In [16]:
# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

# Seaborn for plotting and styling
import seaborn as sns

2019-04-07 16:59:51,982 - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [17]:
import sys
sys.path.insert(0, '../../src/data/')

from config import *

# Load

# Members count

In [18]:
members = pd.read_hdf(analysis_store_path, 'metadata/members')
members.head()

,company,member_id,participates,RecipientID,start_date_ts,end_date_ts
member,,,,,,
1BFLH7NZGE,Company B,535,1,MLRP_0vcgVNdOY7vpB7D,2018-06-12 00:00:00-04:00,2018-08-15 23:59:59-04:00
WLA510JNBN,Company B,537,1,MLRP_74oJw6drZeVi02V,2018-06-19 00:00:00-04:00,2018-08-15 23:59:59-04:00
76B60URM93,Company B,730,1,MLRP_8Cx2w4pmTcBCFUx,2018-06-18 00:00:00-04:00,2018-07-15 23:59:59-04:00
P95P5A6IP2,Company B,536,1,MLRP_5j3Rlvkq0RYG7s1,2018-06-13 00:00:00-04:00,2018-08-15 23:59:59-04:00
MKC03RM7T5,Company N,541,1,MLRP_38gd2aldS3XcP89,2018-06-12 00:00:00-04:00,2018-07-15 23:59:59-04:00


In [19]:
# Count members
companies_master = members.reset_index().groupby('company')\
    .agg({'member':'count'})\
    .rename(columns={'member':'members_count'})

# count potential dyads
companies_master['dyads_count'] = \
    (companies_master.members_count * (companies_master.members_count-1) / 2).astype(int)

# remove EIRs and Staff
companies_master = companies_master.query('company not in ("EIR","Staff")').copy()
companies_master

,members_count,dyads_count
company,,
Company A,4,6
Company B,4,6
Company C,4,6
Company D,2,1
Company E,2,1
Company F,3,3
Company G,4,6
Company H,3,3
Company I,4,6


# create milestone ranking

In [20]:
milestones = pd.read_hdf(performance_clean_store_path,'performance/milestones')
milestones.head()

,m1_meeting1,m1_revenue,m1_rank,m2_meeting1_recovered,m2_meeting2,m2_revenue,m2_rank,m12_meeting1,m12_meeting2,m12_total,...,m3_meeting2_recovered,m3_meeting3,m3_revenue,m3_rank,m123_meeting1,m123_meeting2,m123_meeting3,m123_total,m123_rank,company
0,0.76,"5,285.00",6,0.05,0.79,"5,845",12,0.80,0.79,"11,130.00",...,0.00,0.85,"5,100.00",6,0.80,0.79,0.85,16230.0,12,Company L
1,0.50,"3,500.00",18,0.15,0.50,"4,550",19,0.65,0.50,"8,050.00",...,0.20,0.00,0.00,20,0.65,0.70,0.00,8050.0,20,Company J
2,0.70,"4,900.00",12,0.30,0.75,"7,350",2,1.00,0.75,"12,250.00",...,0.20,0.70,"4,200.00",16,1.00,0.95,0.70,16450.0,10,Company H
3,0.50,"3,500.00",18,0.30,0.65,"6,650",8,0.80,0.65,"10,150.00",...,0.40,0.80,"4,800.00",10,0.80,1.05,0.80,14950.0,14,Company I
4,0.40,"2,800.00",20,0.30,0.40,"4,900",18,0.70,0.40,"7,700.00",...,0.15,0.70,"4,200.00",16,0.70,0.55,0.70,11900.0,19,Company S


In [21]:
companies_panel_milestones = milestones[['company','m1_rank','m2_rank','m3_rank','m123_rank','m123_total']].copy()
companies_panel_milestones.set_index('company', inplace=True)
companies_panel_milestones.sort_index(inplace=True)

companies_panel_milestones_percentile = companies_panel_milestones.rank(pct=True)
companies_panel_milestones_percentile.columns = [x+"_percentile" for x in companies_panel_milestones_percentile.columns.ravel()]
companies_panel_milestones_percentile.head(5)

companies_panel_milestones = companies_panel_milestones.join(companies_panel_milestones_percentile)

In [22]:
companies_panel_milestones.head()

,m1_rank,m2_rank,m3_rank,m123_rank,m123_total,m1_rank_percentile,m2_rank_percentile,m3_rank_percentile,m123_rank_percentile,m123_total_percentile
company,,,,,,,,,,
Company A,2,2,10,3,17750.0,0.175,0.150,0.575,0.15,0.90
Company B,2,9,10,8,16700.0,0.175,0.475,0.575,0.40,0.65
Company C,14,1,19,13,15850.0,0.725,0.050,0.950,0.65,0.40
Company D,7,9,3,5,16950.0,0.425,0.475,0.200,0.25,0.80
Company E,16,13,14,17,14550.0,0.825,0.650,0.725,0.85,0.20


# Add average daily subjective experience

In [23]:
daily_survey_data_clean = pd.read_hdf(surveys_clean_store_path, 'daily/daily_survey_data_clean')
daily_survey_data_clean = daily_survey_data_clean.reset_index()

# add company. note that this drops the researchers answers. which is fine
print('before {}'.format(len(daily_survey_data_clean)))
daily_survey_data_clean = daily_survey_data_clean.join(members['company'], on='member', how='inner')
print('after {}'.format(len(daily_survey_data_clean)))

before 1397
after 1397


In [24]:
companies_panel_subjective = daily_survey_data_clean.groupby('company')[['Q1','Q2']].agg(['mean','count','min','max','var'])
companies_panel_subjective.columns = ["_".join(x) for x in companies_panel_subjective.columns.ravel()]
companies_panel_subjective

,Q1_mean,Q1_count,Q1_min,Q1_max,Q1_var,Q2_mean,Q2_count,Q2_min,Q2_max,Q2_var
company,,,,,,,,,,
Company A,6.017544,57,3.0,7.0,1.338972,6.087719,57,2.0,7.0,1.152882
Company B,5.500000,79,1.0,7.0,1.365385,5.948718,78,3.0,7.0,0.958375
Company C,6.054217,83,3.0,7.0,0.975683,6.295181,83,3.0,7.0,0.719733
Company D,5.046512,43,3.0,7.0,0.997785,5.047619,42,2.0,6.0,0.973287
Company E,5.562500,48,4.0,7.0,0.549202,5.791667,48,4.0,7.0,0.423759
Company F,5.879630,54,2.0,7.0,1.008823,6.203704,54,4.0,7.0,0.580363
Company G,5.888889,81,2.0,7.0,1.075000,5.839506,81,3.0,7.0,1.267670
Company H,4.970149,67,2.0,7.0,1.605156,4.985075,67,1.0,7.0,2.014925
Company I,5.983333,60,2.0,7.0,1.474294,5.900000,60,3.0,7.0,1.820339


# Merge with master panel

In [25]:
companies_panel = companies_master.join(companies_panel_milestones)
companies_panel = companies_panel.join(companies_panel_subjective)
companies_panel

,members_count,dyads_count,m1_rank,m2_rank,m3_rank,m123_rank,m123_total,m1_rank_percentile,m2_rank_percentile,m3_rank_percentile,...,Q1_mean,Q1_count,Q1_min,Q1_max,Q1_var,Q2_mean,Q2_count,Q2_min,Q2_max,Q2_var
company,,,,,,,,,,,,,,,,,,,,,
Company A,4,6,2,2,10,3,17750.0,0.175,0.150,0.575,...,6.017544,57,3.0,7.0,1.338972,6.087719,57,2.0,7.0,1.152882
Company B,4,6,2,9,10,8,16700.0,0.175,0.475,0.575,...,5.500000,79,1.0,7.0,1.365385,5.948718,78,3.0,7.0,0.958375
Company C,4,6,14,1,19,13,15850.0,0.725,0.050,0.950,...,6.054217,83,3.0,7.0,0.975683,6.295181,83,3.0,7.0,0.719733
Company D,2,1,7,9,3,5,16950.0,0.425,0.475,0.200,...,5.046512,43,3.0,7.0,0.997785,5.047619,42,2.0,6.0,0.973287
Company E,2,1,16,13,14,17,14550.0,0.825,0.650,0.725,...,5.562500,48,4.0,7.0,0.549202,5.791667,48,4.0,7.0,0.423759
Company F,3,3,1,7,6,1,17875.0,0.050,0.350,0.350,...,5.879630,54,2.0,7.0,1.008823,6.203704,54,4.0,7.0,0.580363
Company G,4,6,7,2,10,4,17400.0,0.425,0.150,0.575,...,5.888889,81,2.0,7.0,1.075000,5.839506,81,3.0,7.0,1.267670
Company H,3,3,12,2,16,10,16450.0,0.600,0.150,0.825,...,4.970149,67,2.0,7.0,1.605156,4.985075,67,1.0,7.0,2.014925
Company I,4,6,18,8,10,14,14950.0,0.925,0.400,0.575,...,5.983333,60,2.0,7.0,1.474294,5.900000,60,3.0,7.0,1.820339


# Store

In [26]:
with pd.HDFStore(analysis_notebooks_store_path) as store:
    store.put('panels/company_panel',companies_panel,format='table')

# Explore

In [27]:
companies_panel.query("members_count <3")

,members_count,dyads_count,m1_rank,m2_rank,m3_rank,m123_rank,m123_total,m1_rank_percentile,m2_rank_percentile,m3_rank_percentile,...,Q1_mean,Q1_count,Q1_min,Q1_max,Q1_var,Q2_mean,Q2_count,Q2_min,Q2_max,Q2_var
company,,,,,,,,,,,,,,,,,,,,,
Company D,2,1,7,9,3,5,16950.0,0.425,0.475,0.200,...,5.046512,43,3.0,7.0,0.997785,5.047619,42,2.0,6.0,0.973287
Company E,2,1,16,13,14,17,14550.0,0.825,0.650,0.725,...,5.562500,48,4.0,7.0,0.549202,5.791667,48,4.0,7.0,0.423759
Company J,2,1,18,19,20,20,8050.0,0.925,0.975,1.000,...,6.333333,6,5.0,7.0,0.666667,7.000000,6,7.0,7.0,0.000000
Company R,2,1,14,15,18,18,13700.0,0.725,0.800,0.900,...,5.478261,23,3.0,7.0,0.988142,5.782609,23,2.0,7.0,1.268775


In [28]:
companies_panel['m123_total']

company
Company A    17750.0
Company B    16700.0
Company C    15850.0
Company D    16950.0
Company E    14550.0
Company F    17875.0
Company G    17400.0
Company H    16450.0
Company I    14950.0
Company J     8050.0
Company K    16250.0
Company L    16230.0
Company M    16930.0
Company N    16470.0
Company O    14900.0
Company P    16730.0
Company Q    14850.0
Company R    13700.0
Company S    11900.0
Name: m123_total, dtype: float64